In [ ]:
# %%
import os
import requests
from dotenv import load_dotenv

# %%
load_dotenv()

# %%
# URL CORREGIDA - endpoint real de la API, no la documentación
URL = "https://cramer.buk.cl/api/v1/chile/absences/absence"

# %%
# Obtener token de la variable de entorno
TOKEN = os.getenv("TOKEN")
if not TOKEN:
    raise SystemExit("ERROR: define la variable de entorno BUK_AUTH_TOKEN con tu token")
print("Token encontrado exitosamente")

# Encabezados - FORMATO CORRECTO según la imagen
headers = {
    "auth_token": TOKEN,  # Formato correcto de BUK
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# %%
def get_employees(params=None):
    """
    Obtiene datos de inasistencias de BUK con parámetros opcionales
    
    Args:
        params (dict): Parámetros para la consulta, ej: {'from': '2024-01-01', 'to': '2024-12-31'}
    
    Returns:
        dict: Datos de la API o None si hay error
    """
    try:
        # Si no se proporcionan parámetros, usar parámetros por defecto
        if params is None:
            params = {}
        
        print(f"🔗 Consultando URL: {URL}")
        print(f"📋 Parámetros: {params}")
        print(f"🔑 Headers: {headers}")
        
        resp = requests.get(URL, headers=headers, params=params, timeout=10)
        resp.raise_for_status()  # lanza excepción si status != 2xx
        
        print(f"✅ Status Code: {resp.status_code}")
        
        # intentar parsear JSON (si el endpoint devuelve JSON)
        data = resp.json()
        print(f"📊 Tipo de respuesta: {type(data)}")
        
        if isinstance(data, dict) and 'data' in data:
            print(f"📈 Registros encontrados: {len(data['data'])}")
        elif isinstance(data, list):
            print(f"📈 Registros encontrados: {len(data)}")
        
        return data
        
    except requests.HTTPError as e:
        print(f"❌ HTTP error: {e} - status {getattr(e.response, 'status_code', 'N/A')}")
        print(f"   Respuesta del servidor: {getattr(e.response, 'text', 'no body')}")
        return None
    except requests.RequestException as e:
        print(f"❌ Error en la petición: {e}")
        print(f"   Tipo de error: {type(e).__name__}")
        print(f"   URL que falló: {URL}")
        return None
    except ValueError as e:
        print(f"❌ Error al parsear JSON: {e}")
        print(f"   Respuesta recibida: {resp.text[:200]}...")
        return None

# %%
if __name__ == "__main__":
    print("🚀 PROBANDO CONEXIÓN CON LA API BUK")
    print("=" * 50)
    
    # === PRUEBA 1: Sin parámetros ===
    print("\n📋 PRUEBA 1: Consulta sin parámetros")
    empleados = get_employees()
    
    if empleados is not None:
        print("✅ Conexión exitosa!")
        # muestra de forma legible (puedes ajustar según estructura)
        import json
        print("\n🔍 Muestra de respuesta:")
        print(json.dumps(empleados, ensure_ascii=False)[:500] + "...")
    
    # === PRUEBA 2: Con parámetros de fecha === NO FUNCIONA CON FECHAS PERSONALIZADAS ===
    print("\n" + "="*50)
    print("📋 PRUEBA 2: Consulta con parámetros de fecha")
    
    # Ejemplo de parámetros que podrías usar
    parametros_ejemplo = {
        'from': '2024-01-01 00:00:00',
        'to': '2024-08-25 23:59:59'
    }
    
    empleados_con_params = get_employees(params=parametros_ejemplo)
    
    if empleados_con_params is not None:
        print("✅ Consulta con parámetros exitosa!")
    
    # === OTROS EJEMPLOS DE PARÁMETROS POSIBLES ===
    print("\n💡 EJEMPLOS DE PARÁMETROS QUE PUEDES USAR:")
    ejemplos_params = [
        {"from": "2024-01-01", "to": "2024-12-31"},
        {"employee_id": 12345},
        {"status": "approved"},
        {"type": "vacation"},
        {"page": 1, "per_page": 50}
    ]
    
    for i, ejemplo in enumerate(ejemplos_params, 1):
        print(f"   {i}. {ejemplo}")
    
    print(f"\n🔧 Para usar parámetros personalizados:")
    print(f"   params = {{'from': '2024-01-01 00:00:00', 'to': '2024-12-31 23:59:59'}}")
    print(f"   resultado = get_employees(params=params)")

Token encontrado exitosamente
🚀 PROBANDO CONEXIÓN CON LA API BUK

📋 PRUEBA 1: Consulta sin parámetros
🔗 Consultando URL: https://cramer.buk.cl/api/v1/chile/absences/absence
📋 Parámetros: {}
🔑 Headers: {'auth_token': 'JVZhehjdRWdmx2reD5mkKRHA', 'Accept': 'application/json', 'Content-Type': 'application/json'}
✅ Status Code: 200
📊 Tipo de respuesta: <class 'dict'>
📈 Registros encontrados: 25
✅ Conexión exitosa!

🔍 Muestra de respuesta:
{"pagination": {"next": "https://cramer.buk.cl/api/v1/chile/employees/absences/absences?page=2", "previous": null, "count": 25, "total_pages": 25}, "data": [{"id": 85311, "start_date": "2001-10-01", "end_date": "2001-10-01", "days_count": 1.0, "day_percent": 1, "contribution_days": 0.0, "workday_stage": "full_working_day", "application_date": "2001-10-01", "application_end_date": "2001-10-01", "justification": "falta", "employee_id": 4129, "status": "approved", "created_at": "2021-10-22T22:39:36...

📋 PRUEBA 2: Consulta con parámetros de fecha
🔗 Consultand